# Effect of anisotropy on extracellular potentials

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import LFPy
from brainsignals.plotting_convention import mark_subplots, cmap_v_e
from brainsignals.neural_simulations import return_hay_cell

np.random.seed(12345)

# Create a grid of measurement locations, in (um)
grid_x, grid_z = np.mgrid[-400:401:10, -370:1200:10]

grid_y = np.ones(grid_x.shape) * 0

# Define electrode parameters
grid_elec_params = {
    'x': grid_x.flatten(),  # electrode positions
    'y': grid_y.flatten(),
    'z': grid_z.flatten(),
    'method': 'linesource'
}

ax_lfp_dict = dict(aspect=1, frameon=False, xticks=[], yticks=[],
                   ylim=[np.min(grid_z), np.max(grid_z)],
                   xlim=[np.min(grid_x), np.max(grid_x)])


In [ ]:
tstop = 10
dt = 2**-4

syn_height = 150

# Plot single-synapse dipoles
cell = return_hay_cell(tstop=tstop, dt=dt, make_passive=True)
idx = cell.get_closest_idx(x=0, y=0, z=syn_height)

synapse_parameters = {'e': 0., 'weight': 0.002, 'record_current': True,
                      'syntype': 'Exp2Syn', 'tau1': 1, 'tau2': 3,
                      'idx': idx}

synapse = LFPy.Synapse(cell, **synapse_parameters)
synapse.set_spike_times(np.array([1.]))

cell.simulate(rec_imem=True)


In [ ]:

def plot_grid_LFP(cell, grid_elec_params, grid_x, grid_z,
                  ax, synapses, scale_max=None):

    grid_electrode = LFPy.RecExtElectrode(cell, **grid_elec_params)
    M_elec_ps = grid_electrode.get_transformation_matrix()
    lfp_ = M_elec_ps @ cell.imem * 1000

    max_amp_elec_idx = np.argmax(np.max(np.abs(lfp_), axis=1))
    max_amp_t_idx = np.argmax(np.abs(lfp_[max_amp_elec_idx, :]))

    max_amp_LFP = np.max(np.abs(lfp_))
    if not max_amp_LFP == np.abs(lfp_[max_amp_elec_idx, max_amp_t_idx]):
        raise RuntimeError("Wrong with chosen max value")
    LFP = lfp_[:, max_amp_t_idx].reshape(grid_x.shape)

    num = 15
    levels = np.linspace(0.01, 1, num=num)

    scale_max = np.max(np.abs(LFP)) if scale_max is None else scale_max

    levels_norm = scale_max * np.concatenate((-levels[::-1], levels))

    colors_from_map = [cmap_v_e(i/(len(levels_norm) - 2))
                       for i in range(len(levels_norm) - 1)]
    colors_from_map[num - 1] = (1.0, 1.0, 1.0, 1.0)

    ep_intervals = ax.contourf(grid_x, grid_z, LFP,
                               zorder=2, colors=colors_from_map,
                               levels=levels_norm, extend='both')

    ax.contour(grid_x, grid_z, LFP, colors='k', linewidths=(1), zorder=2,
               levels=levels_norm)
    [ax.plot([cell.x[idx, 0], cell.x[idx, 1]],
                 [cell.z[idx, 0], cell.z[idx, 1]], lw=1, c='gray')
     for idx in range(cell.totnsegs)]
    [ax.plot(cell.x[syn.idx].mean(), cell.z[syn.idx].mean(),
             marker='o', c='b', ms=5)
     for syn in synapses]

    return ep_intervals

scalemax = 1

plt.close("all")
fig = plt.figure(figsize=[4., 4.])
fig.subplots_adjust(top=.92, bottom=0., wspace=0.1, left=0.0, right=1)

sigma_anis = [0.3, 0.3, 0.45]
sigma_iso = 0.3

ax_iso = fig.add_subplot(121,
                         title="isotropic\n"+r"$\sigma_{\rm t}$ = 0.3 S/m", **ax_lfp_dict)

ax_anis = fig.add_subplot(122,
                          title="anisotropic\n"+r"$\sigma_{\rm t}$ = [0.3, 0.3, 0.45] S/m",
                          **ax_lfp_dict)

grid_elec_params["sigma"] = sigma_iso

ep_intervals_iso = plot_grid_LFP(cell, grid_elec_params,
                                 grid_x, grid_z, ax_iso,
                                [synapse], scale_max=scalemax)

grid_elec_params["sigma"] = sigma_anis

ep_intervals_anis = plot_grid_LFP(cell, grid_elec_params, grid_x,
                                 grid_z, ax_anis,
                                [synapse], scale_max=scalemax)

for ax in [ax_iso]:
    ax.plot([200, 200], [1000, 500], lw=2, c='k', clip_on=False)
    ax.text(220, 750, "500 µm", ha='left')

cax = fig.add_axes([0.47, 0.1, 0.01, 0.2], frameon=False)
cbar = fig.colorbar(ep_intervals_iso, cax=cax)
cbar.set_label(r'V$_{\rm e}$ (µV)', labelpad=0)
cbar.set_ticks(np.linspace(-scalemax, scalemax, 5))

axes_to_mark = [ax_iso, ax_anis]
mark_subplots(axes_to_mark, xpos=0.03, ypos=1.05)

fig.savefig("fig_anisotropy_effect.pdf")
